In [3]:
# import
import os
import json
import shutil
import platform
import random
import subprocess
import sys

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from pathlib import Path
from collections import Counter, OrderedDict
from typing import Any, Dict, List, Optional, Tuple
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt

from transformers import DetrImageProcessor, DetrForObjectDetection
import transformers
import pytorch_lightning as pl
from torchmetrics.detection.mean_ap import MeanAveragePrecision


In [ ]:
# 프로젝트 루트 탐색

# 한 번만 찾고 고정
def find_project_root(marker_filename=".project-root"):
    current_dir = os.path.abspath(os.getcwd())
    while True:
        if os.path.isfile(os.path.join(current_dir, marker_filename)):
            return current_dir
        parent_dir = os.path.dirname(current_dir)
        if parent_dir == current_dir:
            raise FileNotFoundError(f"Could not find {marker_filename} in any parent directory.")
        current_dir = parent_dir
        
# 경로/출력 폴더 생성(find_project_root() 호출 후 사용)
def ensure_dir(path):
    # 디렉토리 없으면 생성
    os.makedirs(path, exist_ok=True)

PROJECT_ROOT = find_project_root()
def get_project_path(*paths):
    return os.path.join(PROJECT_ROOT, *paths)

In [ ]:
print("=" * 60)
print("EXPERIMENTAL ENVIRONMENT")
print("=" * 60)

# 하드웨어 정보
print("\n[Hardware]")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"  Device: {device}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"  GPU: {gpu_name}")
    print(f"  GPU Memory: {gpu_memory:.1f} GB")
    print(f"  CUDA Version: {torch.version.cuda}")
    if torch.backends.cudnn.is_available():
        print(f"  cuDNN Version: {torch.backends.cudnn.version()}")

# CPU 정보
print(f"  CPU: {platform.processor()}")
print(f"  CPU Cores: {os.cpu_count()}")

# 소프트웨어 정보
print("\n[Software]")
print(f"  OS: {platform.system()} {platform.release()}")
print(f"  Python: {platform.python_version()}")
print(f"  PyTorch: {torch.__version__}")
print(f"  Torchvision: {torchvision.__version__}")
print(f"  Transformers: {transformers.__version__}")
print(f"  PyTorch Lightning: {pl.__version__}")

print("\n" + "=" * 60)
print("Environment check completed")
print("=" * 60)

In [ ]:
# Experiment Management & Logging

# ============================================================
# Cell: Experiment Manager
# ============================================================

class ExperimentManager:
    """실험 관리 및 자동 로그 저장"""
    
    def __init__(self, config: ConfigDict):
        self.config = config
        
        # Experiment ID 생성 (타임스탬프)
        self.experiment_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # 실험 디렉토리 구조: exp/{model}/{dataset}/{exp_id}
        self.exp_dir = Path(config.experiment.save_dir) / \
                       config.model.arch_name / \
                       config.data.dataset_name / \
                       self.experiment_id
        
        # 하위 디렉토리
        self.config_dir = self.exp_dir / "config"
        self.checkpoint_dir = self.exp_dir / "checkpoints"
        self.tensorboard_dir = self.exp_dir / "tensorboard"
        self.results_dir = self.exp_dir / "results"
        
        # 디렉토리 생성
        self._create_directories()
        
        # Config와 metadata 저장
        self._save_experiment_info()
    
    def _create_directories(self):
        """실험 폴더 생성"""
        for directory in [self.config_dir, self.checkpoint_dir, 
                         self.tensorboard_dir, self.results_dir]:
            directory.mkdir(parents=True, exist_ok=True)
    
    def _save_experiment_info(self):
        """Config 및 메타데이터 저장"""
        # Config 저장
        save_config(self.config, self.config_dir / "config.yaml")
        
        # Metadata 저장
        metadata = {
            'experiment_id': self.experiment_id,
            'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'exp_dir': str(self.exp_dir),
            'model': self.config.model.arch_name,
            'dataset': self.config.data.dataset_name,
            'num_classes': self.config.data.num_classes,
        }
        save_config(metadata, self.config_dir / "metadata.yaml")
    
    def save_results(self, results: Dict[str, Any]):
        """실험 결과 저장"""
        save_config(results, self.results_dir / "results.yaml")
    
    def print_info(self):
        """실험 정보 출력"""
        print(f"\n{'='*70}")
        print(f"🧪 EXPERIMENT SETUP")
        print(f"{'='*70}")
        print(f"  Experiment ID:   {self.experiment_id}")
        print(f"  Name:            {self.config.experiment.name}")
        print(f"  Model:           {self.config.model.arch_name}")
        print(f"  Dataset:         {self.config.data.dataset_name} "
              f"({self.config.data.num_classes} classes)")
        print(f"  Classes:         {', '.join(self.config.data.class_names)}")
        print(f"")
        print(f"  📁 Directories:")
        print(f"     Root:         {self.exp_dir}")
        print(f"     Config:       {self.config_dir}")
        print(f"     Checkpoints:  {self.checkpoint_dir}")
        print(f"     TensorBoard:  {self.tensorboard_dir}")
        print(f"     Results:      {self.results_dir}")
        print(f"{'='*70}\n")

print("✅ Experiment Manager 로드 완료")

In [ ]:
# ==========================
# DETR 데이터셋 클래스 정의
# ==========================

class CocoDetection(torchvision.datasets.CocoDetection):
    """
    DETR 학습을 위한 커스텀 COCO Detection 데이터셋 클래스
    """
    
    def __init__(
        self, 
        img_folder: str, 
        ann_file: str,
        imageprocessor: DetrImageProcessor, 
        train: bool = True
    ):
        super(CocoDetection, self).__init__(img_folder, ann_file)
        
        self.imageprocessor = imageprocessor
        self.train = train
        
        # 학습 시 색상/블러 기반 증강 적용 (bbox 수정 불필요)
        self.augment = (
            T.Compose([
                T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
                T.RandomAdjustSharpness(sharpness_factor=1.5, p=0.2),
            ])
            if train
            else None
        )

    def __getitem__(self, idx: int):
        # PIL 이미지와 COCO 형식의 타겟 읽기
        img, target = super(CocoDetection, self).__getitem__(idx)

        # 학습 시 이미지 증강
        if self.augment is not None:
            img = self.augment(img)

        # DETR 형식으로 이미지와 타겟 전처리
        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        encoding = self.imageprocessor(images=img, annotations=target, return_tensors="pt")
        
        # 배치 차원 제거
        pixel_values = encoding["pixel_values"].squeeze()
        target = encoding["labels"][0]

        return pixel_values, target

In [ ]:
# Data Loader

# =========================
# DataLoader 생성
# =========================

# DETR ImageProcessor 초기화
imageprocessor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

def create_collate_fn(imageprocessor: DetrImageProcessor):
    """배치 collate 함수 (패딩 적용)"""
    def collate_fn(batch):
        pixel_values = [item[0] for item in batch]
        labels = [item[1] for item in batch]
        
        # imageprocessor로 패딩 적용
        encoding = imageprocessor.pad(pixel_values, return_tensors="pt")
        
        return {
            'pixel_values': encoding['pixel_values'],
            'pixel_mask': encoding['pixel_mask'],
            'labels': labels
        }
    return collate_fn
# WLS 

In [ ]:
# DETR Model 

# =========================================
# DETR 모델 로드 (사전학습 모델)
# =========================================

import pytorch_lightning as pl
from torchmetrics.detection.mean_ap import MeanAveragePrecision

class Detr(pl.LightningModule):
    """PyTorch Lightning 모듈로 구현한 DETR 학습 래퍼"""

    def __init__(
        self,
        num_labels: int,
        lr: float = 1e-4,
        lr_backbone: float = 1e-5,
        weight_decay: float = 1e-4,
        score_threshold: float = 0.5,
    ):
        super().__init__()
        self.save_hyperparameters()

        self.model = DetrForObjectDetection.from_pretrained(
            "facebook/detr-resnet-50",
            num_labels=num_labels,
            ignore_mismatched_sizes=True,
        )

        self.map_metric = MeanAveragePrecision(
            box_format="cxcywh", iou_type="bbox", class_metrics=True
        )

    def forward(self, pixel_values, pixel_mask=None):
        return self.model(pixel_values=pixel_values, pixel_mask=pixel_mask)

    def common_step(self, batch, batch_idx):
        pixel_values = batch["pixel_values"]
        pixel_mask = batch.get("pixel_mask")
        labels = [{k: v.to(self.device) for k, v in t.items()} for t in batch["labels"]]

        outputs = self.model(
            pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels
        )
        return outputs.loss, outputs.loss_dict

    def training_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)
        self.log("train_loss", loss, prog_bar=True)
        for k, v in loss_dict.items():
            self.log(f"train_{k}", v.item())
        return loss

    def validation_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)
        self.log("val_loss", loss, prog_bar=True)
        for k, v in loss_dict.items():
            self.log(f"val_{k}", v.item())

        # mAP 계산
        with torch.no_grad():
            labels = [{k: v.to(self.device) for k, v in t.items()} for t in batch["labels"]]
            outputs = self.model(
                pixel_values=batch["pixel_values"], 
                pixel_mask=batch.get("pixel_mask")
            )
            self._update_map(outputs, labels)
        return loss

    def _update_map(self, outputs, labels):
        """mAP 업데이트"""
        probs = outputs.logits.softmax(-1)[..., :-1]
        scores, pred_labels = probs.max(-1)
        pred_boxes = outputs.pred_boxes

        preds = []
        targets = []

        for i in range(pred_boxes.shape[0]):
            keep = scores[i] > self.hparams.score_threshold
            preds.append({
                "boxes": pred_boxes[i][keep].detach().cpu(),
                "scores": scores[i][keep].detach().cpu(),
                "labels": pred_labels[i][keep].detach().cpu(),
            })
            targets.append({
                "boxes": labels[i]["boxes"].detach().cpu(),
                "labels": labels[i]["class_labels"].detach().cpu(),
            })

        if preds:
            self.map_metric.update(preds, targets)

    def on_validation_epoch_end(self):
        metrics = self.map_metric.compute()
        for k, v in metrics.items():
            if torch.is_tensor(v) and v.ndim == 0:
                self.log(f"val_{k}", v, prog_bar=True)
        self.map_metric.reset()

    def configure_optimizers(self):
        param_dicts = [
            {"params": [p for n, p in self.named_parameters() 
                       if "backbone" not in n and p.requires_grad]},
            {"params": [p for n, p in self.named_parameters() 
                       if "backbone" in n and p.requires_grad],
             "lr": self.hparams.lr_backbone},
        ]
        return torch.optim.AdamW(
            param_dicts, lr=self.hparams.lr, weight_decay=self.hparams.weight_decay
        )


In [ ]:
# ============================================================
# Cell: 통합 실험 실행 함수
# ============================================================

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger

def run_experiment(config: ConfigDict):
    """Config 기반 전체 실험 실행"""
    
    # 1. Experiment Manager 생성
    exp_manager = ExperimentManager(config)
    exp_manager.print_info()
    
    # 2. Seed 설정 (재현성)
    pl.seed_everything(config.experiment.seed)
    print(f" Seed set to: {config.experiment.seed}\n")
    
    # 3. ImageProcessor 초기화
    imageprocessor = DetrImageProcessor.from_pretrained(
        config.model.pretrained_path
    )
    print(f" ImageProcessor loaded: {config.model.pretrained_path}\n")
    
    # 4. Dataset 생성 (Registry 사용)
    print(" Creating datasets...")
    train_dataset = create_dataset_from_config(
        config.data.dataset_name, "train", imageprocessor, config
    )
    val_dataset = create_dataset_from_config(
        config.data.dataset_name, "val", imageprocessor, config
    )
    print(f"   Train: {len(train_dataset)} samples")
    print(f"   Val:   {len(val_dataset)} samples\n")
    
    # 5. DataLoader 생성
    print(" Creating dataloaders...")
    collate_fn = create_collate_fn(imageprocessor)
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.data.batch_size,
        shuffle=True,
        num_workers=config.data.num_workers,
        collate_fn=collate_fn
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.data.batch_size,
        shuffle=False,
        num_workers=config.data.num_workers,
        collate_fn=collate_fn
    )
    print(f"   Train batches: {len(train_loader)}")
    print(f"   Val batches:   {len(val_loader)}\n")
    
    # 6. 모델 생성
    print(" Creating model...")
    model = Detr(
        num_labels=config.model.num_labels,
        lr=config.model.learning_rate,
        lr_backbone=config.model.lr_backbone,
        weight_decay=config.model.weight_decay,
        score_threshold=config.model.score_threshold
    )
    print(f"   Model: DETR with {config.model.num_labels} classes\n")
    
    # 7. Callbacks 설정
    checkpoint_callback = ModelCheckpoint(
        dirpath=str(exp_manager.checkpoint_dir),
        filename='best-{epoch:02d}-{val_loss:.2f}',
        save_top_k=config.trainer.checkpoint.save_top_k,
        monitor=config.trainer.checkpoint.monitor,
        mode=config.trainer.checkpoint.mode,
        save_last=config.trainer.checkpoint.save_last
    )
    
    early_stop_callback = EarlyStopping(
        monitor=config.trainer.early_stopping.monitor,
        patience=config.trainer.early_stopping.patience,
        mode=config.trainer.early_stopping.mode
    )
    
    # 8. Loggers 설정
    tensorboard_logger = TensorBoardLogger(
        save_dir=str(exp_manager.tensorboard_dir),
        name="",
        version=""
    )
    
    csv_logger = CSVLogger(
        save_dir=str(exp_manager.exp_dir),
        name="logs"
    )
    
    # 9. Trainer 설정
    trainer = pl.Trainer(
        max_epochs=config.trainer.max_epochs,
        accelerator=config.trainer.accelerator,
        devices=config.trainer.devices,
        precision=config.trainer.precision,
        log_every_n_steps=10,
        accumulate_grad_batches=config.trainer.accumulate_grad_batches,
        gradient_clip_val=config.trainer.gradient_clip_val,
        callbacks=[checkpoint_callback, early_stop_callback],
        logger=[tensorboard_logger, csv_logger],
        enable_progress_bar=True
    )
    
    # 10. 학습 정보 출력
    effective_batch = config.data.batch_size * config.trainer.accumulate_grad_batches
    print(f"{'='*70}")
    print(f" TRAINING CONFIGURATION")
    print(f"{'='*70}")
    print(f"  Max Epochs:          {config.trainer.max_epochs}")
    print(f"  Batch Size:          {config.data.batch_size}")
    print(f"  Gradient Accum:      {config.trainer.accumulate_grad_batches}")
    print(f"  Effective Batch:     {effective_batch}")
    print(f"  Learning Rate:       {config.model.learning_rate}")
    print(f"  LR Backbone:         {config.model.lr_backbone}")
    print(f"  Precision:           {config.trainer.precision}")
    print(f"  Early Stop Patience: {config.trainer.early_stopping.patience}")
    print(f"{'='*70}\n")
    
    # 11. 학습 시작
    print(" Starting training...\n")
    trainer.fit(model, train_loader, val_loader)
    
    # 12. 결과 저장
    results = {
        'experiment_id': exp_manager.experiment_id,
        'best_checkpoint': str(checkpoint_callback.best_model_path),
        'best_val_loss': float(checkpoint_callback.best_model_score) 
                        if checkpoint_callback.best_model_score else None,
        'total_epochs': trainer.current_epoch,
        'config_path': str(exp_manager.config_dir / "config.yaml")
    }
    exp_manager.save_results(results)
    
    # 13. 완료 메시지
    print(f"\n{'='*70}")
    print(f" TRAINING COMPLETED!")
    print(f"{'='*70}")
    print(f"  Experiment ID:   {exp_manager.experiment_id}")
    print(f"  Best Checkpoint: {checkpoint_callback.best_model_path}")
    print(f"  Best Val Loss:   {checkpoint_callback.best_model_score:.4f}")
    print(f"")
    print(f"   All results saved in:")
    print(f"     {exp_manager.exp_dir}")
    print(f"")
    print(f"   View TensorBoard:")
    print(f"     tensorboard --logdir {exp_manager.tensorboard_dir}")
    print(f"{'='*70}\n")
    
    return exp_manager, trainer, model

print("✅ 통합 실험 함수 로드 완료")

In [ ]:
# 저장된 실험 재현 

# Config 로드
config 

# 동일한 설정으로 재실험
exp_manager, trainer, model = run_experiment(config)

In [ ]:
# 체크포인트에서 모델 로드

model = Detr.load_from_checkpoint(
    "exp/detr/TomatOD_COCO_3/20250101_123456/checkpoints/best-epoch=10-val_loss=1.23.ckpt",
    num_labels=3
)